In [ ]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error ,r2_score
import matplotlib.pyplot as plt

In [ ]:
#AdaBoost for Regression (Linear Regression Weak Learner)

In [ ]:
class AdaBoostR2:
    def __init__(self ,n_estimators = 20):
        self.n_estimators = n_estimators
        self.models = []
        self.betas = []

    def fit(self ,X ,y):
        n = len(y)
        w = np.ones(n) / n

        for _ in range(self.n_estimators):
            model = LinearRegression()
            model.fit(X ,y ,sample_weight = w)

            y_pred = model.predict(X)

            error = np.abs(y - y_pred)
            error /= max(error)

            weighted_error = np.sum(w * error)

            if weighted_error >= 0.5 :
                break

            beta = weighted_error / (1 - weighted_error)

            w *= beta ** (1 - error)
            w /= np.sum(w)

            self.models.append(model)
            self.betas.append(beta)

    def predict(self ,X) :
        model_preds = np.array([m.predict(X) for m in self.models])
        betas = np.array(self.betas)

        weights = np.log(1 / betas)
        return np.average(model_preds, axis=0, weights=weights)

In [ ]:
X, y = make_regression(n_samples=500, n_features=5, noise=25, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = AdaBoostR2(n_estimators=20)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("R2 Score:", r2_score(y_test, y_pred))

R2 Score: 0.9518096152276085


In [ ]:
#AdaBoost for Classification (Decision Stump)

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
class AdaBoost:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.models = []
        self.alphas = []

    def fit(self, X, y):
        n = len(y)
        w = np.ones(n) / n

        for _ in range(self.n_estimators):
            stump = DecisionTreeClassifier(max_depth=1)
            stump.fit(X, y, sample_weight=w)

            y_pred = stump.predict(X)
            error = np.sum(w * (y_pred != y))

            if error >= 0.5:
                continue

            alpha = 0.5 * np.log((1 - error) / (error + 1e-10))

            w *= np.exp(-alpha * y * y_pred)
            w /= np.sum(w)

            self.models.append(stump)
            self.alphas.append(alpha)

    def predict(self, X):
        final_pred = sum(
            alpha * model.predict(X)
            for alpha, model in zip(self.alphas, self.models)
        )
        return np.sign(final_pred)

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target

y = np.where(y == 0, -1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
stump = DecisionTreeClassifier(max_depth=1)
stump.fit(X_train, y_train)

stump_pred = stump.predict(X_test)
print("Single Stump Accuracy:", accuracy_score(y_test, stump_pred))

Single Stump Accuracy: 0.8947368421052632


In [ ]:
model = AdaBoost(n_estimators=50)
model.fit(X_train, y_train)

ada_pred = model.predict(X_test)
print("AdaBoost Accuracy:", accuracy_score(y_test, ada_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, ada_pred))

AdaBoost Accuracy: 0.9707602339181286
Confusion Matrix:
[[ 61   2]
 [  3 105]]
